In [1]:
import pandas as pd

df = pd.read_csv('../data_lake/business/precios-diarios.csv')
df

,fecha,precio
0,1995-07-20,1.409435
1,1995-07-21,4.924333
2,1995-07-22,1.269500
3,1995-07-23,0.953083
4,1995-07-24,4.305917
...,...,...
9412,2021-04-26,188.325833
9413,2021-04-27,181.770000
9414,2021-04-28,175.874167
9415,2021-04-29,165.203333


In [2]:
from statsmodels.tsa.seasonal import seasonal_decompose
result_add = seasonal_decompose(df['precio'],model = 'add')
result_add.plot();
result_mul = seasonal_decompose(df['precio'],model = 'mul')
result_mul.plot();

'/app_docker/notebooks'